In [2]:
import pypylon.pylon as py
import matplotlib.pyplot as plt
import numpy as np


In [7]:


def Initcam():
    tlf = py.TlFactory.GetInstance()
    devices = tlf.EnumerateDevices()
    
    mycam = py.InstantCamera(tlf.CreateDevice(devices[0]))
    
    return (mycam)

In [8]:
def opencam(camera):
    camera.Open()
    camera.UserSetSelector ="Default"
    camera.UserSetLoad.Execute()
    camera.PixelFormat = "Mono8"
    camera.ExposureTime = 8.0

In [43]:
def acquire(mycam):
    """result = mycam.GrabOne(100)
    myimg = result.GetArray()"""
    if not mycam.IsGrabbing():
        mycam.StartGrabbing(py.GrabStrategy_LatestImageOnly)
    res = mycam.RetrieveResult(100, py.TimeoutHandling_ThrowException)
    if res.GrabSucceeded():
        myimage = res.GetArray()
    res.Release()
                        
    return(myimage)


In [44]:
cam1.Close()
cam1 = Initcam()
opencam(cam1)

###528 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [46]:

image = acquire(cam1) 
#20.4 ms ± 949 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [47]:
print(type(image))

<class 'numpy.ndarray'>


In [48]:
np.shape(image)

(4600, 5280)

In [53]:
from scipy.ndimage import center_of_mass
def centroid( ndarray):
    """
    this had to be modified from a simple scipy.center_of_mass, as it doesn't track correctly the spot center, most likely
    because scipy.center_of_mass tracks all the reflections detected by the camera, but we are only intereste in the most
    intense one.

    centroid = center_of_mass(ndarray)
    return(np.array(centroid))

    :param ndarray:
    :return:
    """
    max_index = np.unravel_index(np.argmax(ndarray), ndarray.shape)

    x_min, x_max = int(max_index[0]) - 150, int(max_index[0]) + 150
    y_min, y_max = int(max_index[1]) - 150, int(max_index[1]) + 150

    roi = ndarray[x_min:x_max, y_min:y_max]
    back_sub = roi - np.min(roi)
    com_roi = (center_of_mass(back_sub)) # center of mass of the ROI
            #print("this is the Center of Mass ROI: ", com_roi)
    com_global = ((com_roi[0]) + x_min, (com_roi[1]) + y_min) # removed the integer casting, as I'm trying sub-pixel resolution.
            #print(com_global[0], com_global[1])
    return (com_global)

In [54]:
%%timeit
center = centroid(image) #this takes about 110 ms.

6.74 ms ± 85.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
cam1.Close()

In [50]:
test = np.shape(image)
print(test)

(4600, 5280)


In [69]:
alan = np.zeros((test))

In [70]:
print(alan)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [64]:
print(np.shape(alan/2))

(4600, 5280)


In [78]:
%%timeit

alan = np.zeros((4600,5280))
for i in range (0,10):
    alan = alan + acquire(camera1)
    


3.56 s ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
